Extracted from: https://www.scimagomedia.com/index.php

In [2]:
import pandas as pd
import numpy as np
import os
import re
from matplotlib import pyplot as plt
import xml
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import requests

In [3]:
url = "https://www.scimagomedia.com/rankings.php?country=Spain&language=Spanish"
response = requests.get(url)
parsed_html = BeautifulSoup(response.content, "html.parser")

def find_data_rows(tag):
    return tag.name == "tr" and "data-offer" in tag.attrs
tags_with_data = parsed_html.find_all("table", attrs={"class": "medios"})[-1].find_all(find_data_rows)
data = [re.sub("\n+", "\n", x.get_text("\n")).split("\n") for x in tags_with_data]
data = [x[2:-1] if len(x) <= 12 else x[4:-1] for x in data]
keys = (
    "name",
    "category",
    "sub_category",
    "daily_unique_ips_avg",
    "total_unique_ips",
    "variation",
    "sequences_of_visits",
    "visit_duration_avg",
    "pages"
)

In [12]:
traffic = pd.DataFrame(data, columns=keys).drop("variation", axis=1)
traffic.replace("--", np.nan, inplace=True)

In [14]:
numeric_cols = ["daily_unique_ips_avg", "total_unique_ips", "sequences_of_visits", "pages"]
traffic[numeric_cols] = traffic[numeric_cols].apply(lambda col: col.replace(["[.]", "[,]"], ["", "."], regex=True), axis=1)
traffic[numeric_cols] = traffic[numeric_cols].astype(float)
visit_duration_splits = traffic["visit_duration_avg"].str.extract("(?P<hours>[0-9]+):(?P<minutes>[0-9]+):(?P<seconds>[0-9]+)").astype(float)
traffic = pd.concat([traffic.drop("visit_duration_avg", axis=1), 
                     visit_duration_splits.drop("hours", axis=1)
                    ], axis=1)

In [15]:
traffic

,name,category,sub_category,daily_unique_ips_avg,total_unique_ips,sequences_of_visits,pages,minutes,seconds
0,7DIESACTUALITAT.COM,Noticias e Información,Noticias globales y actualidad,434.0,9303.0,17193.0,28929.0,1.0,39.0
1,7TELEVALENCIA.COM,Noticias e Información,Noticias globales y actualidad,7682.0,182966.0,251828.0,312441.0,1.0,4.0
2,7TVANDALUCIA.ES,Entretenimiento,Broadcast,1643.0,37572.0,63859.0,180869.0,1.0,25.0
3,ACTUALIDADRIOJABAJA.COM,Noticias e Información,Noticias globales y actualidad,2510.0,59854.0,84214.0,106288.0,0.0,38.0
4,ACTUALITATVALENCIANA.COM,Noticias e Información,Noticias globales y actualidad,3854.0,98610.0,127268.0,293923.0,1.0,8.0
...,...,...,...,...,...,...,...,...,...
558,XCATALUNYA.CAT,Noticias e Información,Noticias globales y actualidad,15658.0,239224.0,549739.0,646193.0,0.0,37.0
559,XTRADIO.ES,Entretenimiento,Broadcast,56.0,1331.0,1965.0,3589.0,1.0,39.0
560,ZA49.ES,Noticias e Información,Noticias globales y actualidad,4862.0,94374.0,162722.0,217017.0,0.0,33.0
561,ZAMORA3PUNTO0.COM,Noticias e Información,Noticias globales y actualidad,1708.0,46142.0,55974.0,94094.0,1.0,8.0


In [18]:
from datetime import datetime
datetime_str = str(datetime.today()).split(".")[0].replace(":", "_")
file_name = f"../data/spain_audience_total_traffic_{datetime_str}.csv"
traffic.to_csv(file_name)
print(f"Saved at {datetime_str}")

Saved at 2023-07-09 11_08_51


In [19]:
traffic.head()

,name,category,sub_category,daily_unique_ips_avg,total_unique_ips,sequences_of_visits,pages,minutes,seconds
0,7DIESACTUALITAT.COM,Noticias e Información,Noticias globales y actualidad,434.0,9303.0,17193.0,28929.0,1.0,39.0
1,7TELEVALENCIA.COM,Noticias e Información,Noticias globales y actualidad,7682.0,182966.0,251828.0,312441.0,1.0,4.0
2,7TVANDALUCIA.ES,Entretenimiento,Broadcast,1643.0,37572.0,63859.0,180869.0,1.0,25.0
3,ACTUALIDADRIOJABAJA.COM,Noticias e Información,Noticias globales y actualidad,2510.0,59854.0,84214.0,106288.0,0.0,38.0
4,ACTUALITATVALENCIANA.COM,Noticias e Información,Noticias globales y actualidad,3854.0,98610.0,127268.0,293923.0,1.0,8.0


## Extraction through file

In [12]:
#!pip install xlrd
!pip install openpyxl

     -------------------------------------- 250.0/250.0 kB 5.1 MB/s eta 0:00:00


In [13]:
import xlrd
import openpyxl

In [4]:
os.listdir("../data")

['.ipynb_checkpoints',
 'BOE-A-1996-8930-consolidado_ley propiedad intelectual.pdf',
 'BOE-A-1996-8930-consolidado_ley propiedad intelectual_plain text.txt',
 'db.sqlite3',
 'extraction checkpoint.json',
 'region_to_media_urls.json',
 'role prompt_keys extraction.txt',
 'SCImago Media Ranking - Spain - Spanish.xlsx',
 'spain_audience_total_traffic_2023-07-09 11_08_51.csv',
 'sqldiff.exe',
 'sqlite3.exe',
 'sqlite3_analyzer.exe',
 'test full news_okdiario.txt',
 'test full news_xataka.txt',
 'versiones de prompts para extraer keys de noticias.txt']

In [5]:
file_name = 'SCImago Media Ranking - Spain - Spanish.xlsx'

In [22]:
ranking = pd.read_excel(os.path.join("data", file_name)).drop(["Country", 
                                                               "Region", 
                                                               "Language"], axis=1)
ranking

,Media,Domain,Global_rank,Overall
0,Informaciones de cuenca,informacioncuenca.com,4343,9.50
1,Diario Independiente Digital,diarioindependientedigital.com,4281,10.75
2,Navarra norte,navarranorte.es,4224,11.75
3,Diari balear,diaribalear.es,4224,11.75
4,Navarra sur,navarrasur.es,4113,14.25
...,...,...,...,...
184,La Vanguardia,lavanguardia.com,69,74.25
185,El Mundo,elmundo.es,48,76.25
186,El Español,elespanol.com,44,76.75
187,ABC,abc.es,25,78.75


In [17]:
# Null values
ranking.isnull().sum()

Media          0
Domain         0
Country        0
Region         0
Language       0
Global_rank    0
Overall        0
dtype: int64

In [33]:
ranking_global = ranking.sort_values("Global_rank")
ranking_overall = ranking.sort_values("Global_rank")
np.equal(ranking_global, ranking_overall).all()

Media          True
Domain         True
Global_rank    True
Overall        True
dtype: bool

In [35]:
ranking.sort_values("Global_rank", ascending=True, inplace=True)
ranking.head(50)

,Media,Domain,Global_rank,Overall
188,El País,elpais.com,7,83.75
187,ABC,abc.es,25,78.75
186,El Español,elespanol.com,44,76.75
185,El Mundo,elmundo.es,48,76.25
184,La Vanguardia,lavanguardia.com,69,74.25
...,...,...,...,...
4,Navarra sur,navarrasur.es,4113,14.25
3,Diari balear,diaribalear.es,4224,11.75
2,Navarra norte,navarranorte.es,4224,11.75
1,Diario Independiente Digital,diarioindependientedigital.com,4281,10.75


In [44]:
ranking.set_index("Media").head(50)

,Domain,Global_rank,Overall
Media,,,
El País,elpais.com,7,83.75
ABC,abc.es,25,78.75
El Español,elespanol.com,44,76.75
El Mundo,elmundo.es,48,76.25
La Vanguardia,lavanguardia.com,69,74.25
El Periódico de Catalunya,elperiodico.com,71,74.00
La Razón,larazon.es,98,71.75
20 Minutos,20minutos.es,103,71.50
ABC de Sevilla,sevilla.abc.es/,103,71.50


In [41]:
# Save list of media urls from spain
ranking.set_index("Media").Domain.to_json("../data/spain_media_name_to_url.json")